# Data Science Lab

In [259]:
import pandas as pd
import numpy as np

df = pd.read_excel('dataset_2023.xlsx')
df = df.drop('Unnamed: 0', axis=1)

In [260]:
df.head()

,brand_name,model_name,milage,first_registration,duration,gear,monthly_fee,emission_value,consumption,horsepower,kilowatts,fuel_type
0,Skoda,Octavia ŠKODA Combi Style TDI DSG,201 km,03/2023,48 Monat (anpassbar),Automatik,"574,01 €",119 g/km,"5,0 l/100 km",150 PS,110 kW,Diesel
1,Volkswagen,T-Cross VW Life TSI,201 km,03/2023,48 Monat (anpassbar),Manuelle Schaltung,"382,58 €",131 g/km,"6,0 l/100 km",95 PS,70 kW,Benzin
2,Seat,Ibiza Austria Edition,15.000 km,10/2022,48 Monat (anpassbar),Manuelle Schaltung,"239,62 €",120 g/km,"5,0 l/100 km",80 PS,59 kW,Benzin
3,Volkswagen,Polo VW,1 km,01/2023,48 Monat (anpassbar),Manuelle Schaltung,"309,11 €",127 g/km,"6,0 l/100 km",80 PS,59 kW,Benzin
4,Audi,A4 Avant 40 TDI quattro S line,105.301 km,12/2019,48 Monat (anpassbar),Automatik,"587,75 €",138 g/km,"5,0 l/100 km",190 PS,140 kW,Diesel


# Preprocessing

### Date column

In [261]:
from datetime import datetime
from dateutil.relativedelta import relativedelta

def calculate_age(registration_date):
    if registration_date == "Neuwagen":
        return 0
    else:
        today = datetime.now()
        date = datetime.strptime(registration_date, "%m/%Y")
        delta = relativedelta(today, date)
        return (delta.years * 12) + delta.months

### Consumption column

In [262]:
def getConsumption(consumption):
    return str(consumption).split()[0]

In [263]:
def replaceComma(text):
    if "." in str(text): 
        return str("".join(str(text).split("."))).replace(",",".")
    else:
        return str(text).replace(",",".")

def removeLetters(text):
    return "".join(c for c in text if c.isdigit() or c == ".")

def removeThousandPoint(text):
    return str(text).replace(".","")

### For the rest of the column

In [264]:
columnsNumeric = ["milage", "duration", "monthly_fee", "horsepower", "emission_value", "kilowatts", "consumption"]

In [265]:
def preProcess(columnsNumeric, df):
    
    # calculating age
    
    df['first_registration'] = df['first_registration'].apply(calculate_age)
    
    
    # removing unnecesary characters from consumption
    
    df['consumption'] = df['consumption'].apply(getConsumption)
    
    
    for i in range(len(columnsNumeric)):
        
        
        df[columnsNumeric[i]] = df[columnsNumeric[i]].apply(replaceComma)
        df[columnsNumeric[i]] = df[columnsNumeric[i]].apply(removeLetters)
    
    # removing thousand point from milage column
    
    df["milage"] = df["milage"].apply(removeThousandPoint)
    
    
    # replacing "" and "nan" values from the different 

    df = df.replace('', np.nan)
    df = df.replace('nan', np.nan)
    
    
    # create the binary vector based on the "gear" column
    
    df['gear'] = np.where(df['gear'] == 'Automatik', 1.0, 0.0)
    df = df.rename(columns={'gear': 'Automatic_gear'})

    
    # create new column combining name and model
    # ------- might make more sense to keep this seperate, will evaluate once models are complete
    
    #df['model_name'] = df['brand_name'] + ', ' + df['model_name']
    #df = df.drop(columns=['brand_name'])
    
    
    # creating dummy variables for fuel_type and model_name column

    df = pd.get_dummies(df, columns=['fuel_type'])
    df = pd.get_dummies(df, columns=['model_name'])
    
    
    # cast numeric to int
    
    df[df.columns.difference(['brand_name'])] = df[df.columns.difference(['brand_name'])].astype(float)


    return df


In [266]:
df = preProcess(columnsNumeric, df)

In [267]:
df.head(5)

,brand_name,milage,first_registration,duration,Automatic_gear,monthly_fee,emission_value,consumption,horsepower,kilowatts,...,model_name_X3 M Competition Individual,model_name_X3 M40d 48V,model_name_X5 xDrive45e PHEV X-LINE,model_name_Yaris 1.5 VVT-i Hybrid GR Sport KlimaA KAM,model_name_i3 120 Ah *LED SCHEINW.*PDC*KOMFORTPAKET*,model_name_i3 120 Ah *LED SCHEINW.*PDC*KOMFORTPAKET*vermietet,"model_name_i30 1,0 T-GDI GO Plus!","model_name_i30 CW 1,0 T-GDI 48V N-Line","model_name_i30 CW 1,0 T-GDI GO Plus!","model_name_i30 Kombi - PD i-Line Plus 1,6 CRDi c1kp3"
0,Skoda,201.0,1.0,48.0,1.0,574.01,119.0,5.0,150.0,110.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,Volkswagen,201.0,1.0,48.0,0.0,382.58,131.0,6.0,95.0,70.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,Seat,15000.0,6.0,48.0,0.0,239.62,120.0,5.0,80.0,59.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,Volkswagen,1.0,3.0,48.0,0.0,309.11,127.0,6.0,80.0,59.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,Audi,105301.0,40.0,48.0,1.0,587.75,138.0,5.0,190.0,140.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


# Implementation

In [271]:
# importing of libraries

from sklearn.impute import SimpleImputer
from sklearn.preprocessing import MinMaxScaler

import pandas as pd

In [269]:
def implementProc(df):
    
    # for later: not hardcode but make it for every column with missing values
    
    
    # create dictionary of mean values for each brand_name
    
    meansDicCons = df.groupby('brand_name')['consumption'].mean().to_dict()
    meansDicEmis = df.groupby('brand_name')['emission_value'].mean().to_dict()
    
    
    # fill nans with mean of brand_name
    
    df['consumption'] = df.apply(lambda x: meansDicCons[x['brand_name']] if pd.isna(x['consumption']) else x['consumption'], axis=1)
    df['emission_value'] = df.apply(lambda x: meansDicEmis[x['brand_name']] if pd.isna(x['emission_value']) else x['emission_value'], axis=1)

    
    # now turn brand_name to dummy variable
    
    df = pd.get_dummies(df, columns=['brand_name'])
    
    
    # Scale using MinMaxScaler
    
    scaler = MinMaxScaler()
    scaled_data = scaler.fit_transform(df)
    scaled_df = pd.DataFrame(scaled_data, columns=df.columns)
    
    return scaled_df

In [272]:
df = implementProc(df)

In [273]:
df.head()

,milage,first_registration,duration,Automatic_gear,monthly_fee,emission_value,consumption,horsepower,kilowatts,fuel_type_Benzin,...,brand_name_Mazda,brand_name_Mercedes-Benz,brand_name_Mitsubishi,brand_name_Nissan,brand_name_Opel,brand_name_Peugeot,brand_name_Seat,brand_name_Skoda,brand_name_Toyota,brand_name_Volkswagen
0,0.001064,0.0125,0.666667,1.0,0.150908,0.341954,0.285714,0.200000,0.199396,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0
1,0.001064,0.0125,0.666667,0.0,0.064517,0.376437,0.357143,0.077778,0.078550,1.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0
2,0.079407,0.0750,0.666667,0.0,0.000000,0.344828,0.285714,0.044444,0.045317,1.0,...,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0
3,0.000005,0.0375,0.666667,0.0,0.031360,0.364943,0.357143,0.044444,0.045317,1.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0
4,0.557443,0.5000,0.666667,1.0,0.157109,0.396552,0.285714,0.288889,0.290030,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
